# SOME METHODS FOR LABEL EXTRACTION

In [17]:
import jieba
from tqdm import tqdm
import tools

In [18]:
def get_frequency_d(root_path, file_count):
    d = {}
    for i in range(file_count):
        file_name = "text"+str(i)
        with open(root_path+file_name,'r',encoding='utf-8') as f:
            content = f.read()
        d[file_name] = {}
        for words in list(jieba.cut(content, cut_all=False, HMM=True)):
            if tools.is_chinese(words):
                if words not in d[file_name]:
                    d[file_name][words] = 1
                else:
                    d[file_name][words] += 1
    return d

### 1. 分词 + 词频统计 + 汉字过滤

In [39]:
d = get_frequency_d('./texts/', 6)

In [40]:
for i in range(6):
    file_name = 'text'+str(i)
    r = sorted(d[file_name].items(), key=lambda x: x[1], reverse=True)[:20:]
    print(file_name, [x[0] for x in r])
    print()

text0 ['的', '南京', '年', '在', '中国', '为', '是', '上', '和', '了', '都', '文化', '中心', '建康', '以', '世界', '城市', '月', '历史', '有']

text1 ['的', '徐州', '是', '为', '在', '汉', '等', '和', '彭城', '有', '了', '楚王陵', '画像石', '年', '之一', '中国', '文化', '出土', '汉墓', '上']

text2 ['年', '苏州', '郡', '为', '县', '公元', '吴县', '月', '的', '撤销', '苏州市', '在', '属', '专区', '公元前', '常熟', '吴郡', '江南', '日', '元年']

text3 ['上海', '年', '的', '松江', '月', '县', '和', '民国', '为', '上海地区', '上海县', '嘉定', '华亭县', '崇明', '府', '宝山', '江苏省', '日', '租界', '等']

text4 ['年', '北京', '的', '为', '月', '撤销', '北京市', '在', '日', '区', '设立', '后', '幽州', '北平', '元年', '天府', '将', '南京', '民国', '和']

text5 ['深圳', '年', '的', '月', '为', '和', '深圳市', '国务院', '宝安县', '在', '老照片', '南头', '地', '发展', '区', '街道', '经济特区', '多年', '城市', '区域']



### 2. 分词 + 词频统计 + 汉字过滤 + 停用词表

In [48]:
with open('./stopwords/baidu_stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = set([x.strip() for x in f.readlines()])

In [49]:
print('停用词数量', len(stop_words))

停用词数量 1396


In [50]:
d = get_frequency_d('./texts/', 6)

In [52]:
for i in range(6):
    file_name = 'text'+str(i)
    r = []
    for x in sorted(d[file_name].items(), key=lambda x: x[1], reverse=True):
        if len(r) >= 20:
            break
        if x[0] not in stop_words:
            r.append(x)
    print(file_name, [x[0] for x in r])
    print()

text0 ['南京', '年', '中国', '上', '都', '文化', '中心', '建康', '世界', '城市', '月', '历史', '人口', '六朝', '金陵', '后', '经济', '日', '地区', '遗存']

text1 ['徐州', '汉', '彭城', '楚王陵', '画像石', '年', '中国', '文化', '出土', '汉墓', '上', '两汉', '墓', '楚王', '历史', '汉代', '狮子山', '人', '江苏', '东汉']

text2 ['年', '苏州', '郡', '县', '公元', '吴县', '月', '撤销', '苏州市', '属', '专区', '公元前', '常熟', '吴郡', '江南', '日', '元年', '吴', '三年', '置']

text3 ['上海', '年', '松江', '月', '县', '民国', '上海地区', '上海县', '嘉定', '华亭县', '崇明', '府', '宝山', '江苏省', '日', '租界', '属', '川沙', '州', '青浦']

text4 ['年', '北京', '月', '撤销', '北京市', '日', '区', '设立', '后', '幽州', '北平', '元年', '天府', '南京', '民国', '今', '通县', '都', '地方', '县']

text5 ['深圳', '年', '月', '深圳市', '国务院', '宝安县', '老照片', '南头', '发展', '区', '街道', '经济特区', '多年', '城市', '区域', '中国', '设立', '日', '批复', '同意']



### 3. 分词 + TF-IDF + 汉字过滤 + 停用词表

In [58]:
import math

In [57]:
with open('./stopwords/baidu_stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = set([x.strip() for x in f.readlines()])

file_count = 6
d = get_frequency_d('./texts/', file_count)

In [59]:
for i in tqdm(range(file_count)):
    file_name = 'text'+str(i)
    total = sum(d[file_name].values())
    for k, v in d[file_name].items():
        if total == 0:
            continue
        d[file_name][k] = v/total
        appear_count = 0
        for j in range(file_count):
            if k in d['text'+str(j)]: appear_count += 1 
        d[file_name][k] = d[file_name][k]*math.log(file_count/appear_count)

100%|██████████| 6/6 [00:00<00:00, 252.37it/s]


In [61]:
for i in range(6):
    file_name = 'text'+str(i)
    r = []
    for x in sorted(d[file_name].items(), key=lambda x: x[1], reverse=True):
        if len(r) >= 20:
            break
        if x[0] not in stop_words:
            r.append(x)
    print(file_name, [x[0] for x in r])
    print()

text0 ['建康', '南京', '遗存', '规模', '六朝', '北伐', '布局', '世界', '金陵', '经济', '人口', '政治', '东南', '意义', '百万', '定为', '文化', '中心', '年前', '天府']

text1 ['徐州', '彭城', '楚王陵', '画像石', '汉', '汉墓', '墓', '汉代', '狮子山', '出土', '两汉', '徐州市', '彭祖', '三绝', '龟山', '俑', '丶', '楚王', '兵马俑', '艺术']

text2 ['苏州', '吴县', '苏州市', '公元', '昆山', '平江', '专区', '常熟', '吴江', '吴郡', '撤销', '会稽', '吴州', '吴', '次年', '江南', '扬州', '浙西', '无锡', '常熟市']

text3 ['上海', '上海地区', '上海县', '华亭县', '崇明', '松江', '租界', '宝山', '沪', '川沙', '青浦', '金山', '奉贤', '南汇', '嘉定', '市区', '华亭', '划分', '上海市', '吴淞江']

text4 ['北京市', '幽州', '北平', '撤销', '通县', '天府', '燕国', '京兆', '房山', '大兴', '河北省', '北京', '天安门', '燕都', '燕山', '辽', '大都', '元大都', '房山区', '蓟县']

text5 ['深圳', '深圳市', '宝安县', '老照片', '南头', '经济特区', '批复', '罗湖区', '国务院', '管理区', '光明', '新区', '街道', '前身', '广州', '新安县', '南越', '香港', '广东省', '罗湖']



### 4. 分词 + TextRank + 汉字过滤 + 停用词表

In [23]:
with open('./stopwords/baidu_stopwords.txt', 'r', encoding='utf-8') as f:
    stop_words = set([x.strip() for x in f.readlines()])

In [28]:
import numpy as np
import jieba
import jieba.posseg as pseg
 
class TextRank(object):
    
    def __init__(self, sentence, window, alpha, iternum):
        self.sentence = sentence
        self.window = window
        self.alpha = alpha
        self.edge_dict = {} #记录节点的边连接字典
        self.iternum = iternum#迭代次数
 
    #对句子进行分词
    # def cutSentence(self):
    #     jieba.load_userdict('user_dict.txt')
    #     tag_filter = ['a','d','n','v']
    #     seg_result = pseg.cut(self.sentence)
    #     self.word_list = [s.word for s in seg_result if s.flag in tag_filter]
    #     print(self.word_list)
        
    def cutSentence(self):
        self.word_list = list(jieba.cut(self.sentence))
        # print(self.word_list)
 
    #根据窗口，构建每个节点的相邻节点,返回边的集合
    def createNodes(self):
        tmp_list = []
        word_list_len = len(self.word_list)
        for index, word in enumerate(self.word_list):
            if word not in self.edge_dict.keys():
                tmp_list.append(word)
                tmp_set = set()
                left = index - self.window + 1#窗口左边界
                right = index + self.window#窗口右边界
                if left < 0: left = 0
                if right >= word_list_len: right = word_list_len
                for i in range(left, right):
                    if i == index:
                        continue
                    tmp_set.add(self.word_list[i])
                self.edge_dict[word] = tmp_set
 
    #根据边的相连关系，构建矩阵
    def createMatrix(self):
        self.matrix = np.zeros([len(set(self.word_list)), len(set(self.word_list))])
        self.word_index = {}#记录词的index
        self.index_dict = {}#记录节点index对应的词
 
        for i, v in enumerate(set(self.word_list)):
            self.word_index[v] = i
            self.index_dict[i] = v
        for key in self.edge_dict.keys():
            for w in self.edge_dict[key]:
                self.matrix[self.word_index[key]][self.word_index[w]] = 1
                self.matrix[self.word_index[w]][self.word_index[key]] = 1
        #归一化
        for j in range(self.matrix.shape[1]):
            sum = 0
            for i in range(self.matrix.shape[0]):
                sum += self.matrix[i][j]
            for i in range(self.matrix.shape[0]):
                self.matrix[i][j] /= sum
 
    #根据textrank公式计算权重
    def calPR(self):
        self.PR = np.ones([len(set(self.word_list)), 1])
        for i in range(self.iternum):
            self.PR = (1 - self.alpha) + self.alpha * np.dot(self.matrix, self.PR)
 
    #输出词和相应的权重
    def printResult(self):
        word_pr = {}
        for i in range(len(self.PR)):
            word_pr[self.index_dict[i]] = self.PR[i][0]
        # res = sorted(word_pr.items(), key = lambda x : x[1], reverse=True)
        # print(res)
        
        r = []
        for x in sorted(word_pr.items(), key=lambda x: x[1], reverse=True):
            if len(r) >= 20:
                break
            if tools.is_chinese(x[0]) and x[0] not in stop_words:
                r.append(x)
        print([x[0] for x in r])
        print()


In [29]:
file_count = 6

In [31]:
root_path = './texts/'
for i in tqdm(range(file_count)):
    file_name = 'text'+str(i)
    with open(root_path+file_name,'r',encoding='utf-8') as f:
        s = f.read()
    # s = '程序员(英文Programmer)是从事程序开发、维护的专业人员。一般将程序员分为程序设计人员和程序编码人员，但两者的界限并不非常清楚，特别是在中国。软件从业人员分为初级程序员、高级程序员、系统分析员和项目经理四大类。'
    tr = TextRank(s, 3, 0.85, 700)
    tr.cutSentence()
    tr.createNodes()
    tr.createMatrix()
    tr.calPR()
    print(file_name, ' \t')
    tr.printResult()

 17%|█▋        | 1/6 [00:01<00:05,  1.20s/it]

text0  	
['南京', '年', '中国', '都', '文化', '上', '建康', '后', '北伐', '人口', '金陵', '民族', '世界', '都城', '六朝', '发现', '东南', '月', '邑', '城市']



 33%|███▎      | 2/6 [00:07<00:10,  2.60s/it]

text1  	
['徐州', '彭城', '年', '上', '丶', '汉', '都', '历史', '徐州市', '文化', '汉高祖', '楚王', '两汉', '刘邦', '中国', '画像石', '楚王陵', '后', '曾', '彭祖']



 50%|█████     | 3/6 [00:08<00:07,  2.36s/it]

...

text3  	
['上海', '年', '县', '上海地区', '松江', '华亭县', '属', '上海县', '江苏省', '民国', '租界', '后', '今', '月', '地区', '十年', '公元', '一带', '成立', '府']



 83%|████████▎ | 5/6 [00:10<00:01,  1.53s/it]

text4  	
['北京', '年', '后', '幽州', '元年', '撤销', '月', '北京市', '北平', '区', '天府', '日', '燕国', '今', '南京', '燕都', '地方', '县', '都', '划归']



100%|██████████| 6/6 [00:10<00:00,  1.82s/it]

text5  	
['深圳', '年', '月', '深圳市', '宝安县', '南头', '发展', '区', '多年', '街道', '城市', '中国', '上', '国务院', '前', '罗湖区', '中', '创新', '香港', '经济特区']



### 5. LDA

In [65]:
from gensim import corpora, models
import jieba.posseg as jp
import jieba


# 简单文本处理
def get_text(text):
    flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')  # 词性
    stopwords = ('的', '就', '是', '用', '还', '在', '上', '作为')  # 停用词
    words_list = []
    for text in texts:
        words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords]
        words_list.append(words)
    return words_list
 
# 生成LDA模型
def LDA_model(words_list):
    # Dictionary()方法遍历所有的文本，为每个不重复的单词分配一个单独的整数ID，同时收集该单词出现次数以及相关的统计信息
    dictionary = corpora.Dictionary(words_list)
    print(dictionary)
    print('打印查看每个单词的id:')
    print(dictionary.token2id)  # 打印查看每个单词的id
 
    # 将dictionary转化为一个词袋
    # doc2bow()方法将dictionary转化为一个词袋。得到的结果corpus是一个向量的列表，向量的个数就是文档数。
    # 在每个文档向量中都包含一系列元组,元组的形式是（单词 ID，词频）
    corpus = [dictionary.doc2bow(words) for words in words_list]
    print('输出每个文档的向量:')
    print(corpus)  # 输出每个文档的向量
 
    # LDA主题模型
    # num_topics -- 必须，要生成的主题个数。
    # id2word    -- 必须，LdaModel类要求我们之前的dictionary把id都映射成为字符串。
    # passes     -- 可选，模型遍历语料库的次数。遍历的次数越多，模型越精确。但是对于非常大的语料库，遍历太多次会花费很长的时间。
    lda_model = models.ldamodel.LdaModel(corpus=corpus, num_topics=2, id2word=dictionary, passes=10)
 
    return lda_model

In [55]:
topic_num = 2

In [68]:
with open('./texts/text0', 'r', encoding='utf-8') as f:
    texts = f.readlines()

# 分词
words_list = get_text(texts)

# LDA模型
lda_model = LDA_model(words_list)
topic_words = lda_model.print_topics(num_topics=topic_num, num_words=5) # 可以用 print_topic 和 print_topics 方法来查看主题
print('\n')

print('打印所有主题，每个主题显示5个词:')
print(topic_words)
print('\n')

# 输出该主题的的词及其词的权重 
print('输出各个主题的词及其词的权重:')
for i in range(topic_num):
    print('TOPIC', i, ':', lda_model.show_topic(i, 20))

Dictionary(499 unique tokens: ['中原', '中国', '临安', '为行', '之俊彦']...)
打印查看每个单词的id:
{'中原': 0, '中国': 1, '临安': 2, '为行': 3, '之俊彦': 4, '之灾': 5, '也': 6, '亦': 7, '人物': 8, '仍定': 9, '价值': 10, '休养生息': 11, '会': 12, '佳境': 13, '先生': 14, '兵燹': 15, '具有': 16, '刘宋': 17, '初立': 18, '动员': 19, '华夏': 20, '南京': 21, '占领': 22, '即将': 23, '历史': 24, '又': 25, '受益': 26, '古都': 27, '合法性': 28, '图': 29, '在于': 30, '地位': 31, '地理位置': 32, '基础': 33, '太平天国': 34, '宋高宗': 35, '定行': 36, '尤': 37, '屡屡': 38, '山川': 39, '异族': 40, '恢复': 41, '患难': 42, '惜': 43, '文学': 44, '无意': 45, '时': 46, '显匡复': 47, '曾多次': 48, '朱': 49, '比较': 50, '民国': 51, '民族': 52, '气象': 53, '洛阳': 54, '燕京': 55, '瓦砾': 56, '皆': 57, '相共': 58, '立志': 59, '统治': 60, '罹祸': 61, '群臣': 62, '舆论': 63, '荒烟': 64, '萧梁': 65, '视为': 66, '议以': 67, '迫于': 68, '选择': 69, '通常': 70, '遭受': 71, '都': 72, '金陵': 73, '长安': 74, '风水': 75, '驱除': 76, '一带': 77, '世界': 78, '东亚': 79, '中华文明': 80, '人': 81, '人种': 82, '人类': 83, '仅': 84, '全世界': 85, '再次': 86, '出土': 87, '化石': 88, '发现': 89, '发祥地': 90, '古人类': 91, '古猿': 9

### 6. Word2Vec + Clustering

### 7. 卡方检验

需要标注数据集

### 8. 基于树模型

需要标注数据集